# [Check if Python Package is installed](https://stackoverflow.com/questions/1051254/check-if-python-package-is-installed)

In [4]:

# https://stackoverflow.com/questions/1051254/check-if-python-package-is-installed
# 
import importlib.util
import sys

# For illustrative purposes.
name = 'pandas'

if name in sys.modules:
    print(f"{name!r} already in sys.modules")
elif (spec := importlib.util.find_spec(name)) is not None:
    # If you choose to perform the actual import ...
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    spec.loader.exec_module(module)
    print(f"{name!r} has been imported")
else:
    print(f"can't find the {name!r} module")

'pandas' has been imported


In [2]:
import pandas as pd
import numpy as np

In [8]:
df = pd.DataFrame(
    {
        "col1": ["a", "a", "b", "b", "a"],
        "col2": [1.0, 2.0, 3.0, np.nan, 5.0],
        "col3": [1.0, 2.0, 3.0, 4.0, 5.0]
    },
    columns=["col1", "col2", "col3"],
)
df

,col1,col2,col3
0,a,1.0,1.0
1,a,2.0,2.0
2,b,3.0,3.0
3,b,NaN,4.0
4,a,5.0,5.0


In [9]:
df2 = df.copy()
df2.loc[0, 'col1'] = 'c'
df2.loc[2, 'col3'] = 4.0
df2

,col1,col2,col3
0,c,1.0,1.0
1,a,2.0,2.0
2,b,3.0,4.0
3,b,NaN,4.0
4,a,5.0,5.0


In [10]:
df.compare(df2)

col1       col3      
  self other self other
0    a     c  NaN   NaN
2  NaN   NaN  3.0   4.0

In [11]:
df.compare(df2, align_axis=0)

col1  col3
0 self     a   NaN
  other    c   NaN
2 self   NaN   3.0
  other  NaN   4.0

In [12]:
df.compare(df2, keep_equal=True)

col1       col3      
  self other self other
0    a     c  1.0   1.0
2    b     b  3.0   4.0

In [13]:
df.compare(df2, keep_shape=True, keep_equal=True)

col1       col2       col3      
  self other self other self other
0    a     c  1.0   1.0  1.0   1.0
1    a     a  2.0   2.0  2.0   2.0
2    b     b  3.0   3.0  3.0   4.0
3    b     b  NaN   NaN  4.0   4.0
4    a     a  5.0   5.0  5.0   5.0

# [Comparing Pandas Dataframes To One Another](https://towardsdatascience.com/comparing-pandas-dataframes-to-one-another-c26853d7dda7)

In [3]:
array_1 = np.array([['LeBron',3],
                    ['Kobe',5],
                    ['Michael',6,],
                    ['Larry',3],
                    ['Magic',5],
                    ['Tim',4]])
df_1 = pd.DataFrame(array_1, 
                    columns=['Player','Rings'])
# Data from friend
array_2 = np.array([['LeBron',3],
                    ['Kobe',3],
                    ['Michael',6,],
                    ['Larry',5],
                    ['Magic',5],
                    ['Tim',4]])
df_2 = pd.DataFrame(array_2, 
                    columns=['Player','Rings'])

In [4]:
df_1.eq(df_2)

,Player,Rings
0,True,True
1,True,False
2,True,True
3,True,False
4,True,True
5,True,True


In [5]:
# Note that we specify the axis to let Pandas know that we care
# about equality across all the columns in a row
df_2[df_1.eq(df_2).all(axis=1)==False]

,Player,Rings
1,Kobe,3
3,Larry,5


In [6]:
df_2[df_1.ne(df_2).any(axis=1)]

,Player,Rings
1,Kobe,3
3,Larry,5


In [7]:
array_3 = np.array([['LeBron',3],
                    ['Kobe',3],
                    ['Michael',6,],
                    ['Larry',5],
                    ['Magic',5],
                    ['Tim',4],
                    ['KG',1],
                    ['Charles',0]])
df_3 = pd.DataFrame(array_3, 
                    columns=['Player','Rings'])

In [8]:
df_1.ne(df_3)

,Player,Rings
0,False,False
1,False,True
2,False,False
3,False,True
4,False,False
5,False,False
6,True,True
7,True,True


In [10]:
df_3[df_1.ne(df_3).any(axis=1)]

,Player,Rings
1,Kobe,3
3,Larry,5
6,KG,1
7,Charles,0


In [11]:
df_4 = pd.DataFrame(array_3,
                    index=['a','b','c','d','e','f','g','h'],
                    columns=['Player','Rings'])
print(df_4)

    Player Rings
a   LeBron     3
b     Kobe     3
c  Michael     6
d    Larry     5
e    Magic     5
f      Tim     4
g       KG     1
h  Charles     0


In [12]:
df_3_reset = df_3.reset_index(drop=True)
# Use boolean indexing and .ne method on reset index
df_3_reset[df_1.ne(df_3_reset).any(axis=1)]

,Player,Rings
1,Kobe,3
3,Larry,5
6,KG,1
7,Charles,0


In [13]:
array_new = np.array([['LeBron',3],
                      ['Michael',6,],
                      ['Magic',5],
                      ['KG',1],
                      ['Charles',0],
                      ['Stephen',3],
                      ['Patrick',0]])
df_new = pd.DataFrame(array_new, 
                      columns=['Player','Rings'])

In [14]:
df_new[df_1.eq(df_new).all(axis=1)]

,Player,Rings
0,LeBron,3


In [15]:
df_new[df_1.ne(df_new).any(axis=1)]

,Player,Rings
1,Michael,6
2,Magic,5
3,KG,1
4,Charles,0
5,Stephen,3
6,Patrick,0


In [17]:
# Need to rename Rings since we are merging on it but we want
# it to show as different columns post-merge
temp = df_new.rename({'Rings': 'Rings_new'}, axis=1)
merged = temp.merge(df_1, how='left', 
                    left_on=['Player','Rings_new'],
                    right_on=['Player','Rings'])
merged

,Player,Rings_new,Rings
0,LeBron,3,3
1,Michael,6,6
2,Magic,5,5
3,KG,1,NaN
4,Charles,0,NaN
5,Stephen,3,NaN
6,Patrick,0,NaN


In [19]:
df_new[merged['Rings'].isna()]

,Player,Rings
3,KG,1
4,Charles,0
5,Stephen,3
6,Patrick,0


In [21]:
final_df = pd.concat([df_1,
                      df_new[merged['Rings'].isna()]],
                     axis=0)
final_df

,Player,Rings
0,LeBron,3
1,Kobe,5
2,Michael,6
3,Larry,3
4,Magic,5
5,Tim,4
3,KG,1
4,Charles,0
5,Stephen,3
6,Patrick,0


# [Compare two DataFrames and output their differences side-by-side](https://stackoverflow.com/questions/17095101/compare-two-dataframes-and-output-their-differences-side-by-side)

In [38]:
df1 = pd.DataFrame(
    {
        "id":[111,112,113,114],
        "Name":['Jack', 'Nick', 'Zoe','Tom'],
        "score":[2.17,1.11,4.12,3.33],
        "isEnrolled":[True,False,True,False],
        "Comment":["He was late to class","Graduated","","Marked"]
    },
    columns=["id", "Name", "score","isEnrolled","Comment"],
)
df1

,id,Name,score,isEnrolled,Comment
0,111,Jack,2.17,True,He was late to class
1,112,Nick,1.11,False,Graduated
2,113,Zoe,4.12,True,


In [39]:
df2 = pd.DataFrame(
    {
        "id":[111,112,113,115],
        "Name":['Jack', 'Nick', 'Zoe','Smith'],
        "score":[2.17,1.21,4.12,2.22],
        "isEnrolled":[True,False,False,True],
        "Comment":["He was late to class","Graduated","On Vocation","Meeting"]
    },
    columns=["id", "Name", "score","isEnrolled","Comment"],
)
df2

,id,Name,score,isEnrolled,Comment
0,111,Jack,2.17,True,He was late to class
1,112,Nick,1.21,False,Graduated
2,113,Zoe,4.12,False,On Vocation


In [25]:
ne = (df1 != df2).any(1)
ne

0    False
1     True
2     True
dtype: bool

In [26]:
ne_stacked = (df1 != df2).stack()
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
changed

id  col       
1   score         True
2   isEnrolled    True
    Comment       True
dtype: bool

In [27]:
difference_locations = np.where(df1 != df2)
changed_from = df1.values[difference_locations]
changed_to = df2.values[difference_locations]
pd.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)

from           to
id col                          
1  score       1.11         1.21
2  isEnrolled  True        False
   Comment           On Vocation

In [28]:
df1.compare(df2)

score       isEnrolled        Comment             
   self other       self  other    self        other
1  1.11  1.21        NaN    NaN     NaN          NaN
2   NaN   NaN       True  False          On Vocation

In [29]:
df1.compare(df2, keep_equal=True, keep_shape=True) 

id        Name       score       isEnrolled                      Comment  \
  self other  self other  self other       self  other                  self   
0  111   111  Jack  Jack  2.17  2.17       True   True  He was late to class   
1  112   112  Nick  Nick  1.11  1.21      False  False             Graduated   
2  113   113   Zoe   Zoe  4.12  4.12       True  False                         

                         
                  other  
0  He was late to class  
1             Graduated  
2           On Vocation

In [30]:
df1.compare(df2, align_axis='index')

score isEnrolled      Comment
1 self    1.11        NaN          NaN
  other   1.21        NaN          NaN
2 self     NaN       True             
  other    NaN      False  On Vocation

In [31]:
def diff_pd(df1, df2):
    """Identify differences between two pandas DataFrames"""
    assert (df1.columns == df2.columns).all(), \
        "DataFrame column names are different"
    if any(df1.dtypes != df2.dtypes):
        "Data Types are different, trying to convert"
        df2 = df2.astype(df1.dtypes)
    if df1.equals(df2):
        return None
    else:
        # need to account for np.nan != np.nan returning True
        diff_mask = (df1 != df2) & ~(df1.isnull() & df2.isnull())
        ne_stacked = diff_mask.stack()
        changed = ne_stacked[ne_stacked]
        changed.index.names = ['id', 'col']
        difference_locations = np.where(diff_mask)
        changed_from = df1.values[difference_locations]
        changed_to = df2.values[difference_locations]
        return pd.DataFrame({'from': changed_from, 'to': changed_to},
                            index=changed.index)

In [32]:
import sys
if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

DF1 = StringIO("""id   Name   score                    isEnrolled           Comment
111  Jack   2.17                     True                 "He was late to class"
112  Nick   1.11                     False                "Graduated"
113  Zoe    NaN                     True                  " "
""")
DF2 = StringIO("""id   Name   score                    isEnrolled           Comment
111  Jack   2.17                     True                 "He was late to class"
112  Nick   1.21                     False                "Graduated"
113  Zoe    NaN                     False                "On vacation" """)
df1 = pd.read_table(DF1, sep='\s+', index_col='id')
df2 = pd.read_table(DF2, sep='\s+', index_col='id')
diff_pd(df1, df2)

from           to
id  col                          
112 score       1.11         1.21
113 isEnrolled  True        False
    Comment           On vacation

In [33]:
import sys
if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO
import pandas as pd

DF1 = StringIO("""id   Name   score                    isEnrolled           Comment
111  Jack   2.17                     True                 "He was late to class"
112  Nick   1.11                     False                "Graduated"
113  Zoe    NaN                     True                  " "
""")
DF2 = StringIO("""id   Name   score                    isEnrolled           Comment
111  Jack   2.17                     True                 "He was late to class"
112  Nick   1.21                     False                "Graduated"
113  Zoe    NaN                     False                "On vacation" """)

df1 = pd.read_table(DF1, sep='\s+', index_col='id')
df2 = pd.read_table(DF2, sep='\s+', index_col='id')
#%%
dictionary = {1:df1,2:df2}
df=pd.concat(dictionary)
df.drop_duplicates(keep=False)

Name  score  isEnrolled      Comment
  id                                       
1 112  Nick   1.11       False    Graduated
  113   Zoe    NaN        True             
2 112  Nick   1.21       False    Graduated
  113   Zoe    NaN       False  On vacation

In [40]:
df_all = pd.concat([df1.set_index('id'), df2.set_index('id')], 
                   axis='columns', keys=['First', 'Second'])
df_all

First                                        Second                   \
     Name score isEnrolled               Comment   Name score isEnrolled   
id                                                                         
111  Jack  2.17       True  He was late to class   Jack  2.17       True   
112  Nick  1.11      False             Graduated   Nick  1.21      False   
113   Zoe  4.12       True                          Zoe  4.12      False   

                           
                  Comment  
id                         
111  He was late to class  
112             Graduated  
113           On Vocation

In [41]:
df_final = df_all.swaplevel(axis='columns')[df.columns[1:]]
df_final

score        isEnrolled                      Comment                      
    First Second      First Second                 First                Second
id                                                                            
111  2.17   2.17       True   True  He was late to class  He was late to class
112  1.11   1.21      False  False             Graduated             Graduated
113  4.12   4.12       True  False                                 On Vocation

In [43]:
def highlight_diff(data, color='yellow'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('First', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

df_final.style.apply(highlight_diff, axis=None)